In [1]:
import os
os.path.expanduser('~/.keras.models')
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tqdm.notebook import tqdm
import pickle

In [2]:
model_conv = VGG16(weights='imagenet', input_shape=(150, 150, 3), include_top=False)

In [3]:
model_conv.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [4]:
import os
from PIL import Image
folder_path = 'PetImages'
extensions = []
for fldr in tqdm(os.listdir(folder_path)):
    sub_folder_path = os.path.join(folder_path, fldr)
    for filee in os.listdir(sub_folder_path):
        file_path = os.path.join(sub_folder_path, filee)
        print('** Path: {}  **'.format(file_path), end="\r", flush=True)
        im = Image.open(file_path)
        rgb_im = im.convert('RGB')
        if filee.split('.')[1] not in extensions:
            extensions.append(filee.split('.')[1])

  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
import os
from PIL import Image
folder_path = 'PetImagesTest'
extensions = []
for fldr in tqdm(os.listdir(folder_path)):
    sub_folder_path = os.path.join(folder_path, fldr)
    for filee in os.listdir(sub_folder_path):
        file_path = os.path.join(sub_folder_path, filee)
        print('** Path: {}  **'.format(file_path), end="\r", flush=True)
        im = Image.open(file_path)
        rgb_im = im.convert('RGB')
        if filee.split('.')[1] not in extensions:
            extensions.append(filee.split('.')[1])

  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
data_generator = ImageDataGenerator(
    rescale=1.0/255.0,
)
generator = data_generator.flow_from_directory(
    'PetImages/',
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

Found 2000 images belonging to 2 classes.


In [7]:
data_generator_test = ImageDataGenerator(
    rescale=1.0/255.0,
)
generator_test = data_generator_test.flow_from_directory(
    'PetImagesTest/',
    target_size=(150, 150),
    batch_size=10,
    class_mode='binary'
)

Found 1000 images belonging to 2 classes.


In [8]:
from tensorflow.keras import models, layers, optimizers

In [9]:
model_conv.trainable = False

In [10]:
model = models.Sequential()
model.add(model_conv)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

In [11]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 256)               2097408   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 16,812,353
Trainable params: 2,097,665
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
history = model.fit_generator(generator, steps_per_epoch=100, epochs=30, validation_data=generator_test, validation_steps=30)

C:\Users\Hacker\AppData\Local\Temp\ipykernel_2052\1337939151.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(generator, steps_per_epoch=100, epochs=30, validation_data=generator_test, validation_steps=30)


Epoch 1/30
 96/100 [===========================>..] - ETA: 4s - loss: 0.5675 - accuracy: 0.7703

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'], color='green')
plt.plot(history.history['val_accuracy'], color='red')